In [6]:
import pandas as pd
import tester
from trainer import read_tables
import os
import argparse
import json
from IPython.display import display, clear_output
import time


In [7]:
def get_args(dataset):
    work_dir=os.path.abspath('..')
    test_args = argparse.Namespace(work_dir=work_dir,
                                   query_dir='demo_query',
                                   dataset=dataset)
    return test_args

In [8]:
def create_query_file(data_dir, question):
    file_name = 'fusion_query.jsonl'
    data_file = os.path.join(data_dir, file_name)
    query_data = {
        "id":0,
        "question":question,
        "table_id_lst":['N/A'],
        "answers":["N/A"],
        "ctxs": [{"title": "", "text": "This is a example passage."}]
    }
    with open(data_file, 'w') as f_o:
        f_o.write(json.dumps(query_data) + '\n')
    

In [9]:
def show_results(question, out_dir, table_dict):
    data_file = os.path.join(out_dir, 'pred_None.jsonl')
    with open(data_file) as f:
        item = json.load(f)
    tag_lst = item['tags']
    N = 5
    pred_table_set = set()
    for tag in tag_lst:
        table_id = tag['table_id']
        if table_id not in pred_table_set:
            pred_table_set.add(table_id)
            table = table_dict[table_id]
            show_table(question, table, len(pred_table_set))
        if len(pred_table_set) >= 5:
            return
    
def show_table(question, table, seq_no):
    print("Question, %s" % question)
    print("Top %d table,         %s " % (seq_no, table['documentTitle']))
    col_lst = table['columns']
    col_names = [a['text'] for a in col_lst]
    row_lst = table['rows']
    row_data = []
    for row_item in row_lst:
        cell_lst = row_item['cells']
        row_item = [a['text'] for a in cell_lst]
        row_data.append(row_item)
    df = pd.DataFrame(row_data,columns=col_names)
    display(df)
    

In [10]:
dataset=input('Type the dataset, ')
args = get_args(dataset)
print('Loading tables')
table_dict = read_tables(args.work_dir, dataset)
data_dir = os.path.join(args.work_dir, 'data', dataset, 'demo_query', 'test')
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)
question = input('Type a question (q to quit),\n').strip()
while question != 'q':
    create_query_file(data_dir, question)
    out_dir = tester.main(args)
    clear_output(wait=False)
    show_results(question, out_dir, table_dict)
    time.sleep(1)
    question = input('Type a question (q to quit),\n').strip()
    

Question, Who won the male 60 metres, male high jump, and female pole vault?
Top 1 table,         List of Greek records in athletics , Men 


,Event,Record,Athlete,Date,Meet,Place,Ref
0,50 m,5.71,Georgios Theodoridis,17 January 1998,-,?,-
1,50 m,5.71,Georgios Theodoridis,25 February 2001,Meeting Pas de Calais,"France Liévin, France",-
2,60 m,6.50,Haralabos Papadias,7 March 1997,World Championships,"France Paris, France",-
3,200 m,20.62,Alexis Alexopoulos,11 February 1996,-,"Greece Piraeus, Greece",-
4,400 m,46.36,Konstantinos Kenteris,13 February 1999,-,"Greece Piraeus, Greece",-
5,800 m,1:48.64,Konstadínos Nakópoulos,28 January 2014,Vienna Indoor Classic,"Austria Vienna, Austria",-
6,1500 m,3:39.63,Panagiotis Papoulias,1 February 1998,-,"Greece Piraeus, Greece",-
7,3000 m,7:43.66,Panagiotis Papoulias,21 February 1998,-,"Greece Piraeus, Greece",-
8,60 m hurdles,7.60,Konstadinos Douvalidis,25 February 2014,Prague Indoor,"Czech Republic Prague, Czech Republic",-
9,High jump,2.35 m,Lambros Papakostas,12 March 1995,World Championships,"Spain Barcelona, Spain",-


Question, Who won the male 60 metres, male high jump, and female pole vault?
Top 2 table,         List of French records in athletics , Women 


,Event,Record,Athlete,Date,Meet,Place,Ref
0,50 m,6.11,Christine Arron,26 February 2006,-,"France Aubière, France",-
1,50 m,6.0 (ht),Annie Alize,15 January 1977,-,"France Grenoble, France",-
2,50 m,6.0 (ht),Laurence Bily,17 January 1985,-,"France Paris, France",-
3,60 m,7.06,Christine Arron,26 February 2006,-,"France Aubière, France",-
4,100 m,11.68,Fabienne Feraez,12 February 2005,-,"Finland Tampere, Finland",-
5,200 m,22.49,Muriel Hurtis,14 March 2003,World Championships,"United Kingdom Birmingham, United Kingdom",-
6,300 m,36.46,Floria Gueï,21 February 2016,-,"France Metz, France",-
7,400 m,51.44,Marie-José Pérec,18 February 1996,Meeting Pas de Calais,"France Liévin, France",-
8,800 m,2:00.42,Élisabeth Grousselle,11 March 2006,World Championships,"Russia Moscow, Russia",-
9,1000 m,2:37.87,Patricia Djaté-Taillard,28 February 1997,-,"France Eaubonne, France",-


Question, Who won the male 60 metres, male high jump, and female pole vault?
Top 3 table,         List of Asian Indoor Games records in athletics , Women's records 


...

Question, Who won the male 60 metres, male high jump, and female pole vault?
Top 4 table,         2016 Asian Indoor Athletics Championships , Women 


,Event,Gold,Gold,Silver,Silver,Bronze,Bronze
0,60 metres,Viktoriya Zyabkina (KAZ),7.27 CR,Yuan Qiqi (CHN),7.33,Dutee Chand (IND),7.37
1,400 metres,Kemi Adekoya (BHR),51.67 AR CR,Elina Mikhina (KAZ),53.85,Quách Thị Lan (VIE),55.69
2,800 metres,Marta Hirpato Yota (BHR),2:04.59,Nimali Waliwarsha Arachchige (SRI),2:04.88,Tatyana Neroznak (KAZ),2:06.32
3,1500 metres,Betlhem Desalegn (UAE),4:21.65,Tigist Belay (BHR),4:22.17,Sugandha Kumari (IND),4;29.06
4,3000 metres,Betlhem Desalegn (UAE),8:44.59,Ruth Jebet (BHR),8:47.24,Alia Saeed Mohammed (UAE),8:48.62
5,60 metres hurdles,Anastasiya Soprunova (KAZ),8.17,Anastasiya Pilipenko (KAZ),8.17,Valentina Kibalnikova (UZB),8.32
6,4×400 m relay,Bahrain Salwa Eid Naser Uwaseun Yusuf Jamal Im...,3:35.07 AR,Iran Farzaneh Fasihi Elnaz Kompani Sepideh Tav...,4:06.51,Jordan Rasem Yacoub Hashem Khattab Omar Boshna...,4:10.55
7,High jump,Svetlana Radzivil (UZB),1.92 m,Nadiya Dusanova (UZB),1.88 m,Zheng Xingjuan (CHN),1.84 m
8,Pole vault,Li Ling (CHN),4.70 m AR CR,Ren Mengqian (CHN),4.30 m,Tomomi Abiko (JPN),4.15 m
9,Long jump,Mayookha Johny (IND),6.35 m,Bui Thi Thu Thao (VIE),6.30 m,Olga Rypakova (KAZ),6.22 m


Question, Who won the male 60 metres, male high jump, and female pole vault?
Top 5 table,         Devin Hester , Personal bests 


,Event,Time (seconds),Venue,Date
0,60 meters,6.77,"Syracuse, New York","February 21, 2004"
1,100 meters,10.42,"Piscataway, New Jersey","May 1, 2004"
2,200 meters,22.23,"New York, New York","March 9, 2002"
3,Event,Mark (meters),Venue,Date
4,Long Jump,7.37,"Syracuse, New York","February 21, 2004"


Type a question (q to quit),
q
